In [1]:
import pandas as pd
from py2neo import Graph

In [6]:
file_path = "../OutputDatallm/ODD04_keywords.csv"
df = pd.read_csv(file_path)

# Ces 2 lignes ne devraient plus être necessaire normalement, puisque lors de la génération du nouveau fichier csv, j'y tiens déjà compte
# df['Cible'] = df['Cible'].apply(lambda x: f"Cible 04-{x.split(' ')[1]}")
# df['Indicateur'] = df['Indicateur'].apply(lambda x: f"Indicateur {x}")
print(df.head())

     ODD        Cible           Indicateur          Mots-cles
0  ODD04  Cible 04-01  Indicateur 04-01-01       young people
1  ODD04  Cible 04-01  Indicateur 04-01-01  proficiency level
2  ODD04  Cible 04-01  Indicateur 04-01-01  primary education
3  ODD04  Cible 04-01  Indicateur 04-01-01    gender equality
4  ODD04  Cible 04-01  Indicateur 04-01-01        clean water


In [3]:
graph = Graph("bolt://localhost:7687", auth=("neo4j", "@Martiale01"))

In [4]:
for _, row in df.iterrows():
    od = row['ODD']
    cible = row['Cible']
    indicateur = row['Indicateur']
    mot_cle = row['Mots-cles']

    # J'ai utilisé MERGE au lieu de CREAT afin de créer les nœuds ODD, Cible, Indicateur, Mots-clés sans doublons. Si le noeud existe déjà, il ne sera pas recréé
    graph.run(f"MERGE (o:ODD {{titre: '{od}'}})")
    graph.run(f"MERGE (c:Cible {{titre: '{cible}'}})")
    graph.run(f"MERGE (i:Indicateur {{code: '{indicateur}'}})")
    graph.run(f"MERGE (m:MotCle {{mot: '{mot_cle}'}})")

    # Les relations entre les nœuds
    graph.run(f"""
    MATCH (o:ODD {{titre: '{od}'}}), (c:Cible {{titre: '{cible}'}}), 
          (i:Indicateur {{code: '{indicateur}'}}), (m:MotCle {{mot: '{mot_cle}'}})
    MERGE (o)-[:CONTIENT]->(c)
    MERGE (c)-[:CONTIENT]->(i)
    MERGE (i)-[:CONTIENT]->(m)
    """)